In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
from IPython.display import display 

import spacy

In [4]:
folder = os.path.join(os.getcwd(), "data", "transcripts")
files = [os.path.join(root, f) for root, _, files in os.walk(folder) for f in files if f.endswith(".docx")]
        

In [5]:
from src.pipelinelib.querying import Parser, Queryable

nlp = spacy.load("de_core_news_sm", disable=["ner", "parser"])
parser = Parser(nlp=nlp, metadata_path="./data/transcripts/Kopie von Transkriptionspaare_Daten.xls")

parser.read_from_files(files)

=== Parser: reading from /home/alexander/Projects/sigmund/data/transcripts/Paar 182_T1_IM_FW_corrected.docx ===
=== Parser: reading from /home/alexander/Projects/sigmund/data/transcripts/Paar 27_T1_IM_FW_corrected.docx ===
=== Parser: reading from /home/alexander/Projects/sigmund/data/transcripts/Paar 60_T1_IM_FW_corrected.docx ===
=== Parser: reading from /home/alexander/Projects/sigmund/data/transcripts/Paar 47_T1_IM_FW_corrected.docx ===
=== Parser: reading from /home/alexander/Projects/sigmund/data/transcripts/Paar 58_T1_IM_FW_corrected.docx ===
=== Parser: reading from /home/alexander/Projects/sigmund/data/transcripts/Paar 29_T1_IM_FW_corrected.docx ===
=== Parser: reading from /home/alexander/Projects/sigmund/data/transcripts/Paar 87_T1_IM_FW_corrected.docx ===
=== Parser: reading from /home/alexander/Projects/sigmund/data/transcripts/Paar 81_T1_IM_FW_corrected.docx ===
=== Parser: reading from /home/alexander/Projects/sigmund/data/transcripts/Paar 105_T1_IM_FW_corrected.docx ===

In [6]:
display(parser.frame)

,document_id,paragraph_id,sentence_id,couple_id,speaker,gender,is_depressed_group,depressed_person,text
0,0,1,0,182,B,M,False,None,Hm?
1,0,2,1,182,A,W,False,None,Du solltest doch nicht nachgucken.
2,0,3,2,182,B,M,False,None,Sag nur ``Ich darf den schönen Mädchen nicht n...
3,0,4,3,182,A,W,False,None,"Doch, darfst du."
4,0,6,4,182,B,M,False,None,Also.
...,...,...,...,...,...,...,...,...,...
114,9,87,114,138,B,M,True,W,"Doch, das ist schon was Besonderes, ich hab je..."
115,9,88,115,138,A,W,True,W,Hm .
116,9,88,116,138,A,W,True,W,Ja es gibt viele Erinnerungen die irgendwie di...
117,9,88,117,138,A,W,True,W,Es ist interessant.


In [7]:
from src.pipelinelib.querying import Parser, Queryable
from src.pipelinelib.text_body import TextBody

queryable = Queryable.from_parser(parser)

In [8]:
document_df = queryable.execute(level=TextBody.DOCUMENT)
display(document_df)

=== Queryable is executing on Document level, query = '' ===


,document_id,couple_id,is_depressed_group,depressed_person,text
0,0,182,False,None,Hm?. Du solltest doch nicht nachgucken.. Sag n...
1,1,27,True,W,"Positive Gefühle.. Ja, positive Gefühle können..."
2,2,60,True,W,"Wenn´s positiv geht, dann wird es nicht laut, ..."
3,3,47,True,W,Erzähl mir was.. Also wir müssen die Fragen je...
4,4,58,False,None,Soll ich anfangen?. Ich hab mir was überlegt.....
5,5,29,False,None,"Gut, fangen wir mal an.. Ja, fang du mal an.. ..."
6,6,87,False,None,"Wie lang?. 10 Minuten. Okay, dann fang mal an...."
7,7,81,False,None,Ich dachte wir reden darüber wie wir die erste...
8,8,105,True,W,Jetzt sollen wir uns unterhalten?. Genau.. Al...
9,9,138,True,W,"Am Ammersee.. Dann machen wir es so.. Da, wo m..."


In [9]:
paragraph_df = queryable.execute(level=TextBody.PARAGRAPH)
display(paragraph_df)

=== Queryable is executing on Paragraph level, query = '' ===


,document_id,paragraph_id,couple_id,speaker,gender,is_depressed_group,depressed_person,text
0,0,1,182,B,M,False,None,Hm?
1,0,2,182,A,W,False,None,Du solltest doch nicht nachgucken.
2,0,3,182,B,M,False,None,Sag nur ``Ich darf den schönen Mädchen nicht n...
3,0,4,182,A,W,False,None,"Doch, darfst du."
4,0,6,182,B,M,False,None,Also.
...,...,...,...,...,...,...,...,...
750,9,85,138,B,M,True,W,Okay und das ist dann wenn du so ein Foto schi...
751,9,86,138,A,W,True,W,"Ja, empfindest du das nicht so?"
752,9,87,138,B,M,True,W,"Doch, das ist schon was Besonderes, ich hab je..."
753,9,88,138,A,W,True,W,Hm .. Ja es gibt viele Erinnerungen die irgend...


In [10]:
sentence_df = queryable.execute(level=TextBody.SENTENCE)
display(sentence_df)

queryable.execute(level=TextBody.SENTENCE)

=== Queryable is executing on Sentence level, query = '' ===


,document_id,paragraph_id,sentence_id,couple_id,speaker,gender,is_depressed_group,depressed_person,text
0,0,1,0,182,B,M,False,None,Hm?
1,0,2,1,182,A,W,False,None,Du solltest doch nicht nachgucken.
2,0,3,2,182,B,M,False,None,Sag nur ``Ich darf den schönen Mädchen nicht n...
3,0,4,3,182,A,W,False,None,"Doch, darfst du."
4,0,6,4,182,B,M,False,None,Also.
...,...,...,...,...,...,...,...,...,...
114,9,87,114,138,B,M,True,W,"Doch, das ist schon was Besonderes, ich hab je..."
115,9,88,115,138,A,W,True,W,Hm .
116,9,88,116,138,A,W,True,W,Ja es gibt viele Erinnerungen die irgendwie di...
117,9,88,117,138,A,W,True,W,Es ist interessant.


=== Queryable is executing on Sentence level, query = '' ===


,document_id,paragraph_id,sentence_id,couple_id,speaker,gender,is_depressed_group,depressed_person,text
0,0,1,0,182,B,M,False,None,Hm?
1,0,2,1,182,A,W,False,None,Du solltest doch nicht nachgucken.
2,0,3,2,182,B,M,False,None,Sag nur ``Ich darf den schönen Mädchen nicht n...
3,0,4,3,182,A,W,False,None,"Doch, darfst du."
4,0,6,4,182,B,M,False,None,Also.
...,...,...,...,...,...,...,...,...,...
114,9,87,114,138,B,M,True,W,"Doch, das ist schon was Besonderes, ich hab je..."
115,9,88,115,138,A,W,True,W,Hm .
116,9,88,116,138,A,W,True,W,Ja es gibt viele Erinnerungen die irgendwie di...
117,9,88,117,138,A,W,True,W,Es ist interessant.


In [11]:
pgs = queryable.by_couple_id(couple_id=27) \
    .is_depressed(d=False) \
    .execute(level=TextBody.DOCUMENT)
display(pgs)

=== Queryable is executing on Document level, query = '(couple_id == 27) and (is_depressed_group == False)' ===


,document_id,couple_id,is_depressed_group,depressed_person,text


'\npipeline  = Pipeline(queryable=queryable)\npipeline.add_component(Tokenizer())\ndisplay(pipeline.execute())\n'

In [12]:
from src.sigmund.preprocessing.words import Tokenizer, Stemmer, Lemmatizer
from src.sigmund.features.tfidf import FeatureTFIDF
from src.pipelinelib.pipeline import Pipeline

pipeline = Pipeline(queryable=queryable)
pipeline.add_component(Tokenizer())
pipeline.add_component(Stemmer())
pipeline.add_component(Lemmatizer())
pipeline.add_component(FeatureTFIDF())
display(pipeline.execute())

=== Starting pipeline with ['Tokenizer', 'Stemmer', 'Lemmatizer', 'FeatureTFIDF'] ===
Executing Tokenizer
=== Queryable is executing on Document level, query = '' ===
=== Queryable is executing on Paragraph level, query = '' ===
=== Queryable is executing on Sentence level, query = '' ===
Executing Stemmer
Executing Lemmatizer
Executing FeatureTFIDF
=== Finished pipeline execution ===


{<src.pipelinelib.extension.Extension at 0x7fbd3662baf0>:      couple_id speaker gender  is_depressed_group  document_id  paragraph_id  \
 0          182       B      M               False            0             1   
 1          182       A      W               False            0             2   
 2          182       B      M               False            0             3   
 3          182       A      W               False            0             4   
 4          182       B      M               False            0             6   
 ..         ...     ...    ...                 ...          ...           ...   
 114        138       B      M                True            9            87   
 115        138       A      W                True            9            88   
 116        138       A      W                True            9            88   
 117        138       A      W                True            9            88   
 118        138       B      M                True  